In [1]:
import os
import sys
import librosa
import numpy as np
from PyQt5 import QtWidgets
from PyQt5.QtGui import QFont
import joblib

In [2]:
class AccentClassifier(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        self.title = 'Accent Classifier'
        self.width = 600
        self.height = 350
        self.initUI()

    def initUI(self):
        # Create the main window
        self.setWindowTitle(self.title)
        self.setGeometry(200, 200, self.width, self.height)

        # Increase the font size by 50%
        font = QFont()
        font.setPointSizeF(font.pointSizeF() * 1.5)
        self.setFont(font)

        # Create a label for the file input
        self.file_label = QtWidgets.QLabel(self)
        self.file_label.setGeometry(40, 40, 250, 45)
        self.file_label.setText("Select File:")

        # Create a button to select the audio file
        self.select_file_button = QtWidgets.QPushButton(self)
        self.select_file_button.setGeometry(350, 40, 125, 45)
        self.select_file_button.setText("Browse...")
        self.select_file_button.clicked.connect(self.open_file_dialog)

        # Create a button to predict the accent
        self.predict_button = QtWidgets.QPushButton(self)
        self.predict_button.setGeometry(225, 115, 150, 45)
        self.predict_button.setText("Predict")
        self.predict_button.clicked.connect(self.predict_accent)

        # Create a text section to display the result
        self.result_text = QtWidgets.QTextEdit(self)
        self.result_text.setGeometry(40, 185, 520, 130)
        self.result_text.setReadOnly(True)

        # Show the window
        self.show()

    def open_file_dialog(self):
        # Open a file dialog to select an audio file
        filename, _ = QtWidgets.QFileDialog.getOpenFileName(self, "Open Audio File", "", "Audio Files (*.wav)")
        if filename:
            self.file_label.setText(os.path.basename(filename))

    def predict_accent(self):
        # Get the selected audio file and extract its MFCC features
        filename = self.file_label.text()
        if filename == 'No file selected':
            return
        filepath = os.path.join('Datasets/TestSidang', filename)
        audio, sr = librosa.load(filepath)
        audio_preprocessed = self.preprocess_audio(audio)
        mfccs = librosa.feature.mfcc(y=audio_preprocessed, sr=sr, n_mfcc=40)
        # Flatten MFCCs into a 1D array using mean
        mfccs = np.mean(mfccs, axis=1)

        # Reshape MFCCs into a 2D array with a single row
        mfccs = mfccs.reshape(1, -1)

        # Turn the multidimensional array into 1 dimensional array
        flattened_mfcc = mfccs.flatten()
        # Load the trained KNN classifier and make a prediction
        knn = self.load_classifier()
        if not knn:
            return
        prediction = knn.predict([flattened_mfcc])[0]

        prediction_proba = knn.predict_proba([flattened_mfcc])

        # self.result_text.setText(f'The accent of the selected audio file is {prediction}')

        # Make a prediction using the KNN model
        probabilities = knn.predict_proba(flattened_mfcc.reshape(1, -1))
        classes = knn.classes_

         # Display the prediction probabilities for each class in the text section
        result_text = f"The Accent is {prediction}\n"

        result_text += "Prediction Probabilities:\n"

        for class_label, probability in zip(classes, probabilities[0]):
            result_text += f"{class_label}: {probability}\n"

        self.result_text.setText(result_text)

        # Display the prediction in the text box
        # if prediction[0] == 0:
        #     self.prediction_text.setText('The accent of the selected audio file is Lombok.')
        # else:
        #     self.prediction_text.setText('The accent of the selected audio file is Madura.')

    def preprocess_audio(self, audio):
        # Normalize the audio signal
        audio_norm = librosa.util.normalize(audio)
        # Apply pre-emphasis filter
        pre_emphasis = 0.97
        audio_preemph = np.append(audio_norm[0], audio_norm[1:] - pre_emphasis * audio_norm[:-1])
        return audio_preemph

    def load_classifier(self):
        # Load the trained KNN classifier from disk
        try:
            clf = joblib.load('D:/DA/Skripsi/knn_model.pkl')
            return clf
        except:
            self.prediction_text.setText('Error: could not load classifier.')
            return None

In [3]:
if __name__ == '__main__':
    app = QtWidgets.QApplication(sys.argv)
    classifier = AccentClassifier()
    sys.exit(app.exec_())

SystemExit: 0

D:\DA\venv\lib\site-packages\IPython\core\interactiveshell.py:3450: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
